# day 260,day 262,day 264

# lets build two parts:
1. cell mode
2. script mode.

# 1. fulfill the cell mode:

## goals:
1. import the essentials.
2. download the data
3. make custom image folder and apply data augmentations to each image in train_data and test_data.
4. batchify the images
5. create TinyVGG model
6. fit and evaluate the model.
7. save the model with pth extension.

# importing the essentials:

In [1]:
import numpy as np
import torch
import torchvision
import os,requests,zipfile
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,classification_report
from mlxtend.plotting import plot_confusion_matrix
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import time

# downloading the data

In [2]:
# creating the directories for storage.
data = Path('data')
images_folder = data / 'images'
images_folder.mkdir(parents=True,exist_ok=True)

# downloading the data:
with open(data / 'pizza_steak_sushi.zip','wb') as f:
  link = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip").content
  f.write(link)


# unzip the file
with zipfile.ZipFile(data/'pizza_steak_sushi.zip','r') as zip:
  zip.extractall(images_folder)
  zip.close()

# create custom image folder

In [3]:
def find_classes(directory):
  class_names = sorted(os.listdir(directory))

  if not class_names:
    raise FileNotFoundError("The {} is not correct. please check!!".format(directory))

  class_idx = {class_names:idx for idx,class_names in enumerate(class_names)}

  return class_names,class_idx

In [4]:
class CustomImageFolder(torch.utils.data.Dataset):

  # initializer
  def __init__(self,directory,transform=None):
    self.image_paths = list(Path(directory).glob('*/*.jpg'))
    self.transform = transform
    self.class_names, self.class_idx = find_classes(directory)

  # load the image:
  def load_image(self,index):
    return Image.open(self.image_paths[index])

  # overwrite the len
  def __len__(self):
    return len(self.image_paths)

  # overwrite the getitem function
  def __getitem__(self,index):
    image = self.load_image(index)
    class_name = self.image_paths[index].parent.stem
    label = self.class_idx[class_name]

    if self.transform:
      return self.transform(image),label
    else:
      return image,label

In [5]:
train_dir = 'data/images/train'
test_dir = 'data/images/test'

train_augmentation = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=[64,64]),
    torchvision.transforms.TrivialAugmentWide(num_magnitude_bins=31),
    torchvision.transforms.ToTensor()
])

test_augmentation = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=[64,64]),
    torchvision.transforms.ToTensor()
])

train_data = CustomImageFolder(directory=train_dir,transform=train_augmentation)
test_data = CustomImageFolder(directory=test_dir,transform=test_augmentation)

# Batchify the train_data and test_data(datasets to dataloader):

In [6]:
train_dataloader = torch.utils.data.DataLoader(train_data,
                                               batch_size=32,
                                               shuffle=True,
                                               num_workers=os.cpu_count())

test_dataloader = torch.utils.data.DataLoader(test_data,
                                              batch_size=32,
                                              shuffle=False,
                                              num_workers=os.cpu_count())

# Create The TinyVGG model

In [7]:
class TinyVGG(torch.nn.Module):
  def __init__(self,i,o,h):
    super().__init__()

    self.conv_block1 = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=i,
                        out_channels=h,
                        kernel_size=3,
                        stride=1,
                        padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels=h,
                        out_channels=h,
                        kernel_size=3,
                        stride=1,
                        padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=3,stride=1)
    )

    self.conv_block2 = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=h,
                        out_channels=h,
                        kernel_size=3,
                        stride=1,
                        padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels=h,
                        out_channels=h,
                        kernel_size=3,
                        stride=1,
                        padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=3,stride=1)
    )

    self.classifier = torch.nn.Sequential(
        torch.nn.Flatten(),
        torch.nn.Linear(in_features=h*3600,out_features=o)
    )

  def forward(self,x):
    return self.classifier(self.conv_block2(self.conv_block1(x)))

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model0 = TinyVGG(i=3,h=10,o=len(train_data.class_names))

# compile the model:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model0.parameters(),
                             lr=0.001)
def accuracy(pred,actual):
  correct = torch.eq(pred,actual).sum().item()
  acc = correct/len(actual)
  return acc

# fit the model
def total_time(start,end):
  total_time = end-start
  print("Total Running TIME: {}".format(total_time))

torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 5
start = time.perf_counter()

trl = []
tsl = []
tra = []
tsa = []

for epoch in range(epochs):

  a_train_loss,a_test_loss,a_train_acc,a_test_acc = 0,0,0,0

  for x_train,y_train in train_dataloader:
    x_train,y_train = x_train.to(device),y_train.to(device)

    train_logits = model0(x_train)
    train_predictions = train_logits.argmax(dim=1)
    train_actuals = y_train
    train_loss = loss(train_logits,train_actuals)
    train_accuracy = accuracy(train_predictions,train_actuals)
    a_train_loss += train_loss
    a_train_acc += train_accuracy

    # zero grad the optimizer
    optimizer.zero_grad()

    # backpropogate the train_loss
    train_loss.backward()

    # step the optimizer up a notch
    optimizer.step()

  # normalizing the accumulated losses and accuracies
  a_train_loss /= len(train_dataloader)
  a_train_acc /= len(train_dataloader)

  # evaluate
  model0.eval()
  with torch.inference_mode():
    for x_test,y_test in test_dataloader:
      x_test,y_test = x_test.to(device),y_test.to(device)

      test_logits = model0(x_test)
      test_predictions = test_logits.argmax(dim=1)
      test_actuals = y_test
      test_loss = loss(test_logits,test_predictions)
      test_accuracy = accuracy(test_predictions,test_actuals)
      a_test_loss += test_loss
      a_test_acc += test_accuracy

    # normalizing the accumulated test loss and test accuracy
    a_test_loss /= len(test_dataloader)
    a_test_acc /= len(test_dataloader)


    # inserting the essentials in the container
    trl.append(a_train_loss)
    tsl.append(a_test_loss)
    tra.append(a_train_acc)
    tsa.append(a_test_acc)

    print("Epoch: {} | Train Loss: {:.3f}, Train Acc: {:.3f} | Test Loss: {:.3f},Test Acc: {:.3f}"
    .format(epoch,a_train_loss,a_train_acc,a_test_loss,a_test_acc))

end = time.perf_counter()
total_time(start,end)





Epoch: 0 | Train Loss: 1.240, Train Acc: 0.383 | Test Loss: 0.452,Test Acc: 0.323
Epoch: 1 | Train Loss: 1.358, Train Acc: 0.246 | Test Loss: 0.819,Test Acc: 0.417
Epoch: 2 | Train Loss: 1.102, Train Acc: 0.414 | Test Loss: 0.752,Test Acc: 0.323
Epoch: 3 | Train Loss: 1.174, Train Acc: 0.281 | Test Loss: 1.040,Test Acc: 0.260
Epoch: 4 | Train Loss: 1.213, Train Acc: 0.305 | Test Loss: 0.831,Test Acc: 0.260
Total Running TIME: 17.466580126000053


# save the model

In [8]:
saved_path = Path('SavedModels')

# make a directory
saved_path.mkdir(parents=True,exist_ok=True)

# creating a name for the model with pth extension
model_name = 'model0.pth'

# creating the model saved path
model_saved_path = saved_path / model_name

# save the model
saved_model = torch.save(obj=model0.state_dict(),
                         f=model_saved_path)

# exploration,analysis and freethrows

In [9]:
train_data.class_names

['pizza', 'steak', 'sushi']

# Part 2 script mode

## goals:
1. write create_dataloader function in a python script file named data_setup.py and import it.
2. write build_model function in a python script and name it build_model and import it

In [10]:
# this is for deleting the going_modular folder incase a new change to be incorporated into the script.
!rm -rf going_modular

# magics script for datasets and dataloader



In [11]:
# making the 'going_modular' directory
folder = Path('going_modular')
folder.mkdir(parents=True,exist_ok=True)


with open(Path(folder / 'data_setup.py'),'w') as f:
  f. write("""
import torch
import torchvision
import os
def create_dataloaders(train_dir,test_dir,train_augmentation,test_augmentation,BatchSize):
  train_data = torchvision.datasets.ImageFolder(root=train_dir,transform=train_augmentation)
  test_data =  torchvision.datasets.ImageFolder(root=test_dir,transform=test_augmentation)
  class_names = train_data.classes
  train_dataloader = torch.utils.data.DataLoader(train_data,
                                                    batch_size=BatchSize,
                                                    shuffle=True,
                                                    num_workers=os.cpu_count(),
                                                    pin_memory=True)
  test_dataloader = torch.utils.data.DataLoader(test_data,
                                                    batch_size=BatchSize,
                                                    shuffle=True,
                                                    num_workers=os.cpu_count(),
                                                  pin_memory=True)
  # pin memory does automatic memory transfer from CPU to GPU at a faster rate.

  return class_names,train_dataloader,test_dataloader"""
  )

In [12]:
# import data_setup file from going_modular folder
from going_modular import data_setup

In [13]:
# create the dataloader and class_names using your scripted file
train_dir = 'data/images/train'
test_dir = 'data/images/test'
class_names, train_dataloader,test_dataloader = data_setup.create_dataloaders(
    train_dir,
    test_dir,
    train_augmentation,
    test_augmentation,
    32
)

class_names,train_dataloader,test_dataloader

(['pizza', 'steak', 'sushi'],
 <torch.utils.data.dataloader.DataLoader at 0x7f673f87a020>)

# turning our model building code into python script

In [14]:
with open(Path(folder / 'model_building.py'),'w') as f:
  f.write(
"""

# Args:
# 1. it creates the TinyVGG model from CNN Explainer website:
# 2. it takes an integer for input(i)_i is number of color channels in the images.
# 3. it takes an integer for hidden_layers(h)
# 4. it takes and integer for outputs(o)_ which is the number of classes

# importing the essentials
import torch
class TinyVGG(torch.nn.Module):
  def __init__(self,i,o,h):
    super().__init__()

    self.conv_block1 = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=i,
                        out_channels=h,
                        kernel_size=3,
                        stride=1,
                        padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels=h,
                        out_channels=h,
                        kernel_size=3,
                        stride=1,
                        padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=3,stride=1)
    )

    self.conv_block2 = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=h,
                        out_channels=h,
                        kernel_size=3,
                        stride=1,
                        padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels=h,
                        out_channels=h,
                        kernel_size=3,
                        stride=1,
                        padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=3,stride=1)
    )

    self.classifier = torch.nn.Sequential(
        torch.nn.Flatten(),
        torch.nn.Linear(in_features=h*3600,out_features=o)
    )

  def forward(self,x):
    return self.classifier(self.conv_block2(self.conv_block1(x)))

 """

  )


In [15]:
# importing the model_building method from the python script
from going_modular import model_building



In [16]:
# create a model
model1 = model_building.TinyVGG(i=3,o=len(class_names),h=10)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model1.to(device)

TinyVGG(
  (conv_block1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=36000, out_features=3, bias=True)
  )
)

# turning our model training code into python script(`engine.py`)

In [21]:
with open(Path(folder / 'engine.py'),'w') as f:
  f.write(
      """

# importing the essentials:

# Args:
# supply model,train_dataloader,and test_dataloader

import torch
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def model_train(model,train_dataloader,test_dataloader):

  # compile the model:
  loss = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(params=model.parameters(),
                              lr=0.001)
  def accuracy(pred,actual):
    correct = torch.eq(pred,actual).sum().item()
    acc = correct/len(actual)
    return acc

  # fit the model
  def total_time(start,end):
    total_time = end-start
    print("Total Running TIME: {}".format(total_time))

  torch.manual_seed(42)
  torch.cuda.manual_seed(42)

  epochs = 5
  start = time.perf_counter()

  trl = []
  tsl = []
  tra = []
  tsa = []
  history = []

  for epoch in range(epochs):

    a_train_loss,a_test_loss,a_train_acc,a_test_acc = 0,0,0,0

    for x_train,y_train in train_dataloader:
      x_train,y_train = x_train.to(device),y_train.to(device)

      train_logits = model(x_train)
      train_predictions = train_logits.argmax(dim=1)
      train_actuals = y_train
      train_loss = loss(train_logits,train_actuals)
      train_accuracy = accuracy(train_predictions,train_actuals)
      a_train_loss += train_loss
      a_train_acc += train_accuracy

      # zero grad the optimizer
      optimizer.zero_grad()

      # backpropogate the train_loss
      train_loss.backward()

      # step the optimizer up a notch
      optimizer.step()

    # normalizing the accumulated losses and accuracies
    a_train_loss /= len(train_dataloader)
    a_train_acc /= len(train_dataloader)

    # evaluate
    model.eval()
    with torch.inference_mode():
      for x_test,y_test in test_dataloader:
        x_test,y_test = x_test.to(device),y_test.to(device)

        test_logits = model(x_test)
        test_predictions = test_logits.argmax(dim=1)
        test_actuals = y_test
        test_loss = loss(test_logits,test_predictions)
        test_accuracy = accuracy(test_predictions,test_actuals)
        a_test_loss += test_loss
        a_test_acc += test_accuracy

      # normalizing the accumulated test loss and test accuracy
      a_test_loss /= len(test_dataloader)
      a_test_acc /= len(test_dataloader)


      # inserting the essentials in the container
      trl.append(a_train_loss)
      tsl.append(a_test_loss)
      tra.append(a_train_acc)
      tsa.append(a_test_acc)

      print("Epoch: {} | Train Loss: {:.3f}, Train Acc: {:.3f} | Test Loss: {:.3f},Test Acc: {:.3f}"
      .format(epoch,a_train_loss,a_train_acc,a_test_loss,a_test_acc))

  end = time.perf_counter()
  total_time(start,end)
  return history.extend([trl,tra,tsl,tsa])
"""

  )

In [22]:
# import the engine module
from going_modular import engine

In [23]:
# train the model with engine.py script that we have created!
engine.model_train(model1,train_dataloader,test_dataloader)

Epoch: 0 | Train Loss: 1.227, Train Acc: 0.379 | Test Loss: 0.469,Test Acc: 0.260
Epoch: 1 | Train Loss: 1.222, Train Acc: 0.293 | Test Loss: 0.961,Test Acc: 0.402
Epoch: 2 | Train Loss: 1.127, Train Acc: 0.305 | Test Loss: 0.989,Test Acc: 0.381
Epoch: 3 | Train Loss: 1.096, Train Acc: 0.301 | Test Loss: 0.963,Test Acc: 0.402
Epoch: 4 | Train Loss: 1.095, Train Acc: 0.348 | Test Loss: 0.979,Test Acc: 0.392
Total Running TIME: 19.157911669999976


# create a python script for saving our model(`utils.py`)

In [29]:
with open(Path(folder / 'utils.py'),'w') as f:
  f.write(
"""
# importing the essentials:
from pathlib import Path
import torch

def save_model(model_name):

  saved_path = Path('SavedModels')

  # make a directory
  saved_path.mkdir(parents=True,exist_ok=True)

  # adding the .pth extension if model_name is provided without it

  if model_name[-4:] != '.pth':
    model_name = model_name + '.pth'

  # creating the model saved path
  model_saved_path = saved_path / model_name

  # save the model
  saved_model = torch.save(obj=model0.state_dict(),
                          f=model_saved_path)

  return saved_model

"""
  )